In [1]:
import os

dir_data = "/content/"
if not os.path.exists(dir_data):
  os.mkdir(dir_data)
os.chdir(dir_data)
!wget http://www.phontron.com/kftt/download/kftt-alignments.tar.gz
!tar -xf kftt-alignments.tar.gz
!rm kftt-alignments.tar.gz

--2022-05-31 17:35:26--  http://www.phontron.com/kftt/download/kftt-alignments.tar.gz
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1281468 (1.2M) [application/gzip]
Saving to: ‘kftt-alignments.tar.gz’

kftt-alignments.tar 100%[===================>]   1.22M  --.-KB/s    in 0.1s    

2022-05-31 17:35:26 (11.0 MB/s) - ‘kftt-alignments.tar.gz’ saved [1281468/1281468]



In [2]:
import os

os.chdir("/")
!git clone https://github.com/hir777/Machine_Translation_Proto.git
os.chdir("/Machine_Translation_Proto")
!pip install -r requirements.txt

corpus_path = "/Machine_Translation_Proto/corpus/"
if not os.path.exists(corpus_path):
  os.mkdir(corpus_path)

fatal: destination path 'Machine_Translation_Proto' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import glob
import os

path = "/content/kftt-alignments/data/"
files = []
for i in range(1, 16):
  files.append(os.path.join(path, "english-00" + str(i) + ".txt" if i < 10 else "english-0" + str(i) + ".txt"))
  print(files[i-1])

with open(corpus_path + 'train.en', 'wb') as f_new:
    for f in files:
        with open(f, 'rb') as f_org:
            f_new.write(f_org.read())

/content/kftt-alignments/data/english-001.txt
/content/kftt-alignments/data/english-002.txt
/content/kftt-alignments/data/english-003.txt
/content/kftt-alignments/data/english-004.txt
/content/kftt-alignments/data/english-005.txt
/content/kftt-alignments/data/english-006.txt
/content/kftt-alignments/data/english-007.txt
/content/kftt-alignments/data/english-008.txt
/content/kftt-alignments/data/english-009.txt
/content/kftt-alignments/data/english-010.txt
/content/kftt-alignments/data/english-011.txt
/content/kftt-alignments/data/english-012.txt
/content/kftt-alignments/data/english-013.txt
/content/kftt-alignments/data/english-014.txt
/content/kftt-alignments/data/english-015.txt


In [8]:
import glob
import os

path = "/content/kftt-alignments/data/"
files = []
for i in range(1, 16):
  files.append(os.path.join(path, "japanese-00" + str(i) + ".txt" if i < 10 else "japanese-0" + str(i) + ".txt"))
  print(files[i-1])

with open(corpus_path + 'train.ja', 'wb') as f_new:
    for f in files:
        with open(f, 'rb') as f_org:
            f_new.write(f_org.read())

/content/kftt-alignments/data/japanese-001.txt
/content/kftt-alignments/data/japanese-002.txt
/content/kftt-alignments/data/japanese-003.txt
/content/kftt-alignments/data/japanese-004.txt
/content/kftt-alignments/data/japanese-005.txt
/content/kftt-alignments/data/japanese-006.txt
/content/kftt-alignments/data/japanese-007.txt
/content/kftt-alignments/data/japanese-008.txt
/content/kftt-alignments/data/japanese-009.txt
/content/kftt-alignments/data/japanese-010.txt
/content/kftt-alignments/data/japanese-011.txt
/content/kftt-alignments/data/japanese-012.txt
/content/kftt-alignments/data/japanese-013.txt
/content/kftt-alignments/data/japanese-014.txt
/content/kftt-alignments/data/japanese-015.txt


In [9]:
import shutil

corpus_path = "/Machine_Translation_Proto/corpus/"
shutil.copyfile(os.path.join(path, "japanese-dev.txt"), os.path.join(corpus_path, "valid.ja"))
shutil.copyfile(os.path.join(path, "english-dev.txt"), os.path.join(corpus_path, "valid.en"))
shutil.copyfile(os.path.join(path, "japanese-test.txt"), os.path.join(corpus_path, "test.ja"))
shutil.copyfile(os.path.join(path, "english-test.txt"), os.path.join(corpus_path, "test.en"))

'/Machine_Translation_Proto/corpus/test.en'

In [11]:
os.chdir("/Machine_Translation_Proto/")
!bash pre-process.sh

+ TRAIN_EN=/Machine_Translation_Proto/corpus/train.en
+ TRAIN_JA=/Machine_Translation_Proto/corpus/train.ja
+ VALID_EN=/Machine_Translation_Proto/corpus/valid.en
+ VALID_JA=/Machine_Translation_Proto/corpus/valid.ja
+ TEST_EN=/Machine_Translation_Proto/corpus/test.en
+ cat /Machine_Translation_Proto/corpus/train.en /Machine_Translation_Proto/corpus/train.ja
+ python src/sp_learn.py --input train.enja --prefix bpe --vocab-size 4000 --character-coverage 0.9995 --threads 1
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: train.enja
  input_format: 
  model_prefix: bpe
  model_type: BPE
  vocab_size: 4000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 1
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_d

In [29]:
!fairseq-train \
    data-bin \
    --fp16 \
    --save-interval 5 \
    --log-interval 1 \
    --log-format simple \
    --max-epoch 150 \
    --update-freq 1 \
    --max-update 30000 \
    --max-tokens 4000 \
    --arch transformer \
    --encoder-normalize-before \
    --decoder-normalize-before \
    --encoder-embed-dim 512 \
    --encoder-ffn-embed-dim 2048 \
    --encoder-attention-heads 8 \
    --encoder-layers 8 \
    --decoder-embed-dim 512 \
    --decoder-ffn-embed-dim 2048 \
    --decoder-attention-heads 8 \
    --decoder-layers 8 \
    --share-all-embeddings \
    --dropout 0.3 \
    --attention-dropout 0.0 \
    --activation-dropout 0.0 \
    --activation-fn relu \
    --optimizer adam \
    --adam-betas '(0.9, 0.999)' \
    --lr 0.0015 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 4000 \
    --warmup-init-lr 1e-07 \
    --clip-norm 1.0 \
    --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.3 \
    | tee train.log

2022-05-31 17:47:12 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=1.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=2048, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=8, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=True, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_bac

In [30]:
!fairseq-interactive data-bin \
    --buffer-size 1024 \
    --batch-size 128 \
    --path checkpoints/checkpoint150.pt \
    --beam 5 \
    --lenpen 0.6 \
    < test.en \
    | grep '^H' \
    | cut -f 3 \
    | python src/decode.py \
    | tee output.txt \
    | sacrebleu corpus/test.ja

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 86.7 94.8/90.7/87.0/83.8 (BP = 0.974 ratio = 0.975 hyp_len = 14077 ref_len = 14442)


In [32]:
!head output.txt corpus/test.ja

==> output.txt <==
足利 義満 ( あしかが よしみつ ) は 室町 幕府 の 第 3 代 征夷 大 将軍 ( 在位 1368 年 - 1394 年 ) で あ る 。
官位 は 従 一 位 太政 大臣 、 贈 正 一位 。
封号 は 日本 国 王 。
幼名 を 春王 と い う 。
父 は 室町 幕府 第 2 代 将軍 ・ 足利 義詮 で 、 母 は 紀 良子 。
正室 は 大 納言 日野 時光 の 娘 で あ る 日野 業子 で 、 後光厳 天皇 の 寵姫 で あ る 日野 宣子 の 介添え で 義満 の 室 と な る 。
その 後 、 業子 の あと は 業子 の 姪 で あ る 日野 康子 が 正室 と な る ( のち の 北山 院 ) 。
側室 は 足利 義持 と 足利 義教 の 生母 ・ 藤原 慶子 、 足利 義嗣 の 生母 ・ 春日 局 ( 足利 義満 側室 ) など 。
義満 が 御所 を 北 小路 室町 へ 移 し た こと に よ り 、 義満 は 「 室町 殿 」 と も 呼 ば れ た 。
のち に 足利 将軍 を 指 す 呼称 と な り 、 政庁 を 兼ね た 将軍 邸 は 後 に 歴史 用語 と し て 「 室町 幕府 」 と 呼 ば れ る こと に な っ た 。

==> corpus/test.ja <==
足利 義満 （ あしかが よしみつ ） は 室町 幕府 の 第 3 代 征夷 大 将軍 （ 在位 1368 年 - 1394 年 ） で あ る 。
官位 は 従 一 位 太政 大臣 、 贈 正 一位 。
封号 は 日本 国 王 。
幼名 を 春王 と い う 。
父 は 室町 幕府 第 2 代 将軍 ・ 足利 義詮 で 、 母 は 紀 良子 。
正室 は 大 納言 日野 時光 の 娘 で あ る 日野 業子 で 、 後光厳 天皇 の 寵姫 で あ る 日野 宣子 の 介添え で 義満 の 室 と な る 。
その 後 、 業子 の あと は 業子 の 姪 で あ る 日野 康子 が 正室 と な る （ のち の 北山 院 ） 。
側室 は 足利 義持 と 足利 義教 の 生母 ・ 藤原 慶子 、 足利 義嗣 の 生母 ・ 春日 局 ( 足利 義満 側室 ) など 。
義満 が 御所 を 北 小路 室

In [34]:
import re
import unicodedata
import sentencepiece as spm
from fairseq.models.transformer import TransformerModel

sp = spm.SentencePieceProcessor(model_file='bpe.model')
model = TransformerModel.from_pretrained('./checkpoints/', checkpoint_file='checkpoint150.pt', data_name_or_path='data-bin')

def preprocess_en(x, alpha=None):
  sp = spm.SentencePieceProcessor()
  sp.Load("/Machine_Translation_Proto/bpe.model")

  x = unicodedata.normalize('NFKC', x)
  x = x.strip()
  x = ' '.join(x.split())
  if alpha is None:
      x = sp.encode(x, out_type = int)
  else:
      x = sp.encode(x, out_type = int, enable_sampling = True, alpha = alpha)
  x = ' '.join([sp.IdToPiece(i) for i in x])
  return x

def translate_x(x):
  x = preproc_en(x)
  x = model.translate(x, beam = 5, lenpen = 0.6)
  x = ''.join(x.split()).replace('▁', '').strip()
  return x

In [25]:
import re
import unicodedata
from sacremoses import MosesTokenizer
import sentencepiece as spm
from fairseq.models.transformer import TransformerModel

mt = MosesTokenizer(lang = 'en')
sp = spm.SentencePieceProcessor(model_file='bpe.model')
model = TransformerModel.from_pretrained('./checkpoints/', checkpoint_file='checkpoint120.pt', data_name_or_path='data-bin')

def preproc_en(x):
  x = unicodedata.normalize('NFKC', x)
  x = re.sub(mt.AGGRESSIVE_HYPHEN_SPLIT[0], r'\1 - ', x)
  x = mt.tokenize(x, escape = False)
  x = ' '.join(x)
  x = x.lower()
  x = ' '.join(sp.encode(x, out_type = 'str'))
  return x

def translate_y(x):
  x = preproc_en(x)
  x = model.translate(x, beam = 5, lenpen = 0.6)
  x = ''.join(x.split()).replace('▁', '').strip()
  return x

In [35]:
while True:
  x = input('英文を入力 > ')
  if not x:
    break
  x = translate_x(x)
  print('翻訳結果 > ' + x)

英文を入力 > I have a pen
翻訳結果 > また入っている。
英文を入力 > his farther was the 2nd shogun , yoshiakira ashikaga , and his mother was ki no yoshiko .
翻訳結果 > 父は室町幕府第2代将軍・足利義詮で、母は紀良子。


KeyboardInterrupt: ignored